# Spacecharge Benchmark - Comparing to Bmad



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pmd_beamphysics import ParticleGroup
from pytao import Tao
from scipy.constants import c

import impact.z as IZ
from impact.tests.z.conftest import bmad_files

In [ ]:
TRACK_START = "BEGINNING"
TRACK_END = "END"

L = 1
BUNCH_CHARGE = 1e-9

NX = 32
NY = 32
NZ = 32
N_PARTICLE = 10_000
DS_STEP = 0.01

BMAD_SC_ON = True  # Note that Impact-Z always has SC on

CHIRP = 0

LATTICE_FILE = bmad_files / "drift.bmad"

E_TOT = 10e6
BETA_A = 1000
BETA_B = 1000

In [ ]:
# Gaussian
def set_gaussian(
    tao,
    n_particle=N_PARTICLE,
    a_emit=1.0e-9,
    b_emit=1.0e-9,
    bunch_charge=1e-9 / 1000,
    sig_pz0=1e-4,
    sig_z=1e-4,
    center_pz=0,
    chirp=0,  # 1/m
):
    sig_pz = np.hypot(sig_pz0, chirp * sig_z)

    cmds = [
        f"set beam_init n_particle = {n_particle}",
        "set beam_init random_engine = quasi",
        "set beam_init saved_at = MARKER::*, BEGINNING, END",
        f"set beam_init a_emit = {a_emit}",
        f"set beam_init b_emit = {b_emit}",
        f"set beam_init bunch_charge = {bunch_charge}",
        f"set beam_init sig_pz = {sig_pz}",
        f"set beam_init sig_z = {sig_z}",
        f"set beam_init dpz_dz = {chirp}",
        f"set beam_init center(6) = {center_pz}",
        "set beam_init random_sigma_cutoff = 4",
    ]
    tao.cmds(cmds)
    tao.cmd("set global lattice_calc_on = T")


def get_particles(tao, ele_id):
    return ParticleGroup(data=tao.bunch_data(ele_id))

In [ ]:
tao = Tao(lattice_file=LATTICE_FILE, plot="mpl")

tao.cmd(f"set ele beginning e_tot = {E_TOT}")
tao.cmd(f"set ele beginning beta_a = {BETA_A}")
tao.cmd(f"set ele beginning beta_b = {BETA_B}")

set_gaussian(tao, n_particle=N_PARTICLE, bunch_charge=BUNCH_CHARGE, chirp=CHIRP)

In [ ]:
tao.cmd("set bmad_com csr_and_space_charge_on = T")
tao.cmd("set space_charge particle_bin_span = 1")
tao.cmd(f"set space_charge n_bin = {NZ}")
tao.cmd(f"set space_charge ds_track_step = {DS_STEP}")
# tao.cmd("set bmad_com radiation_damping_on  = T")  # off by default
# tao.cmd("set bmad_com radiation_fluctuations_on = T")  # off by default

if BMAD_SC_ON:
    tao.cmd("set ele * space_charge_method = fft_3d")
    tao.cmd(f"set space_charge space_charge_mesh_size = {NX} {NY} {NZ}")
else:
    tao.cmd("set ele * space_charge_method = off")
tao.cmd(f"set ele * DS_STEP = {DS_STEP}")

In [ ]:
%%time
tao.cmd("set global track_type = beam; set global track_type = single")

P0 = get_particles(tao, "beginning")
P1 = get_particles(tao, "end")

P0.plot("delta_t", "energy")
P0.plot("x", "y")
P1.plot("delta_t", "energy"), P1["sigma_t"] * c * 1e6

In [ ]:
sigma_z0 = P0["sigma_t"] * c
sigma_z1 = P1["sigma_t"] * c

sigma_z0

In [ ]:
# P0.drift_to_t()
# P0['sigma_x']*1e3, P0['sigma_y']*1e3, P0['sigma_z']*1e3

In [ ]:
sigma_z1 / sigma_z0

# Impact-Z

In [ ]:
input = IZ.ImpactZInput.from_tao(tao, verbose=False, write_beam_eles="")
I = IZ.ImpactZ(input, use_temp_dir=False, workdir="./tmp-sc-bench")
I.initial_particles = P0

In [ ]:
I.input.space_charge_on()
I.input.integrator_type = 1
I.input.nx = NX
I.input.ny = NY
I.input.nz = NZ

In [ ]:
I.input

In [ ]:
%%time
I.nproc = 1
I.run(verbose=False);

In [ ]:
P20 = I.output.particles["initial_particles"]
P2 = I.output.particles["final_particles"]
P2.plot("x", "y")
P2.plot("delta_t", "energy")
P2["sigma_t"] / P20["sigma_t"]

In [ ]:
stats = I.output.stats
mc2 = I.input.reference_particle_mass
z = stats.z
x = stats.mean_x
y = stats.mean_y
sigma_x = I.output.stats.sigma_x
sigma_y = I.output.stats.sigma_y
sigma_z = I.output.stats.sigma_t * c
energy = stats.mean_energy

ix_eles = tao.lat_list("*", "ele.ix_ele")

bparams = pd.DataFrame(list(map(tao.bunch_params, ix_eles)))


x_tao = tao.bunch_comb("x")
y_tao = tao.bunch_comb("y")
sigma_x_tao = tao.bunch_comb("x.sigma")
sigma_y_tao = tao.bunch_comb("y.sigma")
sigma_z_tao = tao.bunch_comb("z.sigma")
p_tao = (1 + tao.bunch_comb("pz")) * tao.bunch_comb("p0c")
energy_tao = np.hypot(p_tao, mc2)
s_tao_raw = tao.bunch_comb("s")
s_tao = s_tao_raw - s_tao_raw[0]


# s_tao2 = bparams.s - bparams.s[0]
# sigma_x_tao2 = np.sqrt(bparams.sigma_11)
# sigma_y_tao2 = np.sqrt(bparams.sigma_33)
# sigma_z_tao2 = np.sqrt(bparams.sigma_55)

fig, axes = plt.subplots(3, figsize=(12, 8))

ax = axes[0]
ax.plot(z, x * 1e6, label="Impact-Z")
ax.plot(s_tao, x_tao * 1e6, "--", label="Tao")
ax.set_ylabel(r"$\left<x\right>$ (µm)")

ax = axes[1]
ax.plot(z, sigma_x * 1e6, label="Impact-Z")
# ax1.plot(s_tao, sigma_x_tao, "--", label="Tao")
ax.plot(s_tao, sigma_x_tao * 1e6, "--", label="Tao")
ax.set_ylabel(r"$\sigma_x$ (µm)")
# ax1.set_yscale('log')

ax = axes[2]
ax.plot(z, sigma_z * 1e6, label="Impact-Z")
ax.plot(s_tao, sigma_z_tao * 1e6, "--", label="Tao")
ax.set_ylabel(r"$\sigma_z$ (µm)")

ax.legend()


s_max = s_tao.max()
for ax in axes:
    ax.set_xlim(0, s_max)

ax.set_xlabel(r"$s$ (m)")

## Compare

In [ ]:
def compare(
    xkey="x",
    ykey="y",
):
    fig, ax = plt.subplots()

    plist = (
        (P1, "Bmad", "X", "blue"),
        (P2, "Impact-Z", ".", "orange"),
    )

    for p, label, marker, color in plist:
        ax.scatter(p[xkey], p[ykey], label=label, marker=marker, alpha=0.5, color=color)

    for p, label, marker, color in plist:
        ax.scatter(
            p["mean_" + xkey],
            p["mean_" + ykey],
            marker="+",
            color=color,
            facecolor="black",
        )

    plt.legend()

    ax.set_xlabel(xkey)
    ax.set_ylabel(ykey)


compare()

In [ ]:
P1["mean_x"], P2["mean_x"]

In [ ]:
compare("delta_t", "energy")

In [ ]:
compare("delta_t", "xp")

In [ ]:
energy0 = P0["mean_energy"]
P1["mean_energy"] - energy0, P2["mean_energy"] - energy0

# Impact-Z Lorentz integrator 

`ImpactZInput.integrator_type = 2`

Note that this is not expected to work as well because the dipole fringe fields are not able to be specified.

In [ ]:
I2 = I.copy()

In [ ]:
I2.input.integrator_type = 2
I2.run(verbose=False)

P3 = I2.output.particles["final_particles"]

In [ ]:
stats1 = I.output.stats
stats2 = I2.output.stats

eref = I.output.stats.energy_ref[0]  # should be constant

fig, ax = plt.subplots(1, figsize=(12, 4))

ax.plot(stats1.z, (stats1.mean_energy - eref) / 1e6, label="Impact-T, map")
ax.plot(stats2.z, (stats2.mean_energy - eref) / 1e6, label="Impact-T, nonlinear")

ax.plot(s_tao, (energy_tao - eref) / 1e6, label="Bmad")

ax.set_xlabel(r"$s$ (m)")
ax.set_ylabel(r"$dE$ (MeV)")

plt.legend()

In [ ]:
def compare(
    xkey="x",
    ykey="y",
    skip=10,
):
    fig, ax = plt.subplots()

    for p, label, marker in (
        (P1, "Bmad", "X"),
        (P2, "Impact-Z map", "."),
        (P3, "Impact-Z nonlinear", "."),
    ):
        ax.scatter(
            p[xkey][::skip], p[ykey][::skip], label=label, marker=marker, alpha=0.5
        )

    plt.legend()

    ax.set_xlabel(xkey)
    ax.set_ylabel(ykey)


compare("delta_t", "xp")

In [ ]:
compare("delta_t", "energy")